In [1]:
!which python

/Users/ahmadabdullahtariq/Documents/Projects/Thesis/.venv/bin/python


In [2]:
import pandas as pd
import numpy as np 
import plotly.graph_objects as go

In [14]:
df = pd.read_pickle('../dataset/preprocessed/284_imputed.pkl')

print(df.isna().sum())

df['Definitive'] = df['Definitive'].astype('int')
df['Rooms'] = df['Rooms'].astype('int')

print(df.dtypes)

TargetDate      0
Definitive      0
Rooms           0
SnapshotDate    0
LAR             0
dtype: int64
TargetDate      datetime64[ns]
Definitive               int64
Rooms                    int64
SnapshotDate    datetime64[ns]
LAR                    float64
dtype: object


# K Nearest Neighbor

In [15]:
df.head()

,TargetDate,Definitive,Rooms,SnapshotDate,LAR
0,2018-01-01,58,290,2017-10-05,76.05
1,2018-01-01,58,290,2017-10-06,76.05
2,2018-01-01,58,290,2017-10-08,76.05
3,2018-01-01,63,290,2017-10-12,76.05
4,2018-01-01,62,290,2017-10-13,76.05


In [16]:
# consider data for 2018 and 2019
df = df[df['TargetDate']<='2019-12-31']
# creating new columns
df['LeadTime'] = (df['TargetDate']-df['SnapshotDate']).dt.days
df['DOW'] = df.TargetDate.dt.dayofweek
# df = df.drop(columns=['SnapshotDate'])
# just rearrange columns 
df = df[['TargetDate','SnapshotDate','DOW','LeadTime','LAR','Rooms','Definitive']]

In [17]:
df.head()

,TargetDate,SnapshotDate,DOW,LeadTime,LAR,Rooms,Definitive
0,2018-01-01,2017-10-05,0,88,76.05,290,58
1,2018-01-01,2017-10-06,0,87,76.05,290,58
2,2018-01-01,2017-10-08,0,85,76.05,290,58
3,2018-01-01,2017-10-12,0,81,76.05,290,63
4,2018-01-01,2017-10-13,0,80,76.05,290,62


In [18]:
dx = df.pivot(index='TargetDate',columns='LeadTime',values='Definitive')
dx = dx.ffill(axis=1).bfill(axis=1)
dx['DOW'] = dx.index.dayofweek

In [32]:
index = int(len(dx)*0.75)
train = dx[:index]
test  = dx[index:]

In [34]:
y_train = train.pop(0)
y_test  = test.pop(0)

X_train = train
X_test  = test

In [27]:
from sklearn.neighbors import KNeighborsClassifier

In [40]:
neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=3)

In [53]:
y_pred = neigh.predict(X_test.values)
# print(neigh.predict_proba([[0.9]]))

In [54]:
X_test['True'] = y_test
X_test['Pred'] = y_pred

In [55]:
X_test

LeadTime,1,2,3,4,5,6,7,8,9,10,...,83,84,85,86,87,88,89,DOW,True,Pred
TargetDate,,,,,,,,,,,,,,,,,,,,,
2019-07-03,275.0,275.0,252.0,240.0,214.0,198.0,181.0,172.0,164.0,163.0,...,37.0,37.0,35.0,34.0,33.0,33.0,31.0,2,281.0,244.0
2019-07-04,262.0,255.0,244.0,224.0,205.0,184.0,165.0,151.0,146.0,139.0,...,46.0,45.0,45.0,43.0,42.0,41.0,41.0,3,267.0,226.0
2019-07-05,273.0,275.0,271.0,251.0,246.0,235.0,230.0,220.0,211.0,208.0,...,104.0,101.0,100.0,100.0,98.0,96.0,96.0,4,275.0,267.0
2019-07-06,281.0,274.0,278.0,274.0,257.0,252.0,243.0,242.0,232.0,226.0,...,129.0,128.0,123.0,121.0,121.0,119.0,117.0,5,285.0,274.0
2019-07-07,244.0,239.0,238.0,239.0,236.0,223.0,219.0,213.0,210.0,204.0,...,111.0,110.0,109.0,106.0,105.0,105.0,106.0,6,243.0,250.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-27,271.0,271.0,269.0,267.0,268.0,268.0,267.0,269.0,265.0,265.0,...,49.0,49.0,49.0,47.0,46.0,45.0,45.0,4,275.0,273.0
2019-12-28,276.0,273.0,274.0,275.0,273.0,273.0,273.0,280.0,283.0,279.0,...,58.0,57.0,56.0,56.0,55.0,54.0,53.0,5,283.0,284.0
2019-12-29,254.0,249.0,247.0,237.0,230.0,224.0,223.0,221.0,230.0,231.0,...,72.0,70.0,68.0,68.0,68.0,66.0,65.0,6,258.0,245.0


In [56]:
# temp = X_test[X_test['TargetDate']=='2019-09-27']
fig = go.Figure()
fig.add_trace(go.Scatter(x=X_test.index, y=X_test['True'],mode='markers',name='True'))
fig.add_trace(go.Scatter(x=X_test.index, y=X_test['Pred'],mode='markers',name='Pred'))
fig.show()   